In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Setting up PHD Filter
This portion pulls and builds the filter code. If you have the filter code pulled, make sure to navigate "cd" into that directory.

In [ ]:
!git clone https://github.com/MobileRoboticistsW21/PHD-object-traking.git

In [ ]:
%cd PHD-object-traking/

In [ ]:
!git checkout main   # can change this to main if branch is merged 
!git pull

In [ ]:
%cd /content/PHD-object-traking/
!bash setup.sh
!mkdir build
%cd build

In [ ]:
!cmake ..
!make phd_json_dump

In [ ]:
!./phd_json_dump
# enter this: ../data/first_20_frame_optical_res.json

# Visualization and Testing

Be sure to set the correct data paths for the code to run as expected.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import json
from google.colab.patches import cv2_imshow
import cv2
import numpy as np

In [ ]:
# filtered_data_dir = "/content/drive/My Drive/PhD_FIlter_Miracle_runs/PHD-object-traking/output_data/run_output_MOT_16_14.json"
# detection_data_dir = "/content/drive/My Drive/PhD_FIlter_Miracle_runs/PHD-object-traking/data/MOT16-14_optical_flow_res.json"
# images_loc = os.path.join('/content/drive/My Drive/Mask_RCNN_stuff/', 'MOT16-14')

images_dir = "/content/drive/My Drive/Mask_RCNN_stuff/MOT16-14"
filtered_data_dir = "output_data/run_output_MOT_16_14.json"
detection_data_dir = "data/MOT16-14_optical_flow_res.json"
ground_truth_dir = ""

In [ ]:
with open(filtered_data_dir, encoding='UTF-8', errors='ignore') as inputFile:
    data_gen = json.load(inputFile)
with open(detection_data_dir, encoding='UTF-8', errors='ignore') as inputFile:
    data_input = json.load(inputFile)
images = os.listdir(images_loc)

In [ ]:
for i in range(len(data_gen)):             # number of frames
    bbs = data_gen[i]['bb']
    weights = data_gen[i]['weights']

    plt.clf(); plt.xlim(0, 540); plt.ylim(0, 960);
    ax = plt.axes()
    for bb in bbs:
        rect = patches.Rectangle((bb[0] - bb[2] / 2, bb[1] - bb[3] / 2), bb[2], bb[3], linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
    # plt.show()
    plt.pause(0.000001);


In [ ]:
from python_utils.id_assigner import assign_ids_postfact
data = assign_ids_postfact(data)

with open('/content/PHD-object-traking/build/run_phd_output_extract_with_ids.json', 'w') as outfile:
    json.dump(data, outfile, indent=4)

In [ ]:
from python_utils.box_vis_utils import xywh_to_corners

# NOTE I AM offseting the images to 1 on purpose because optical flow
Length = 20

for image,in_data, gen_data in zip(images[1:Length+1], data_input[:Length], data_gen[:Length]):             # number of frames
  image = cv2.imread(os.path.join(images_loc, images[0]))
  # I am assuming that data is still outputting xywh formatted data
  data_tlbr_in = xywh_to_corners(in_data['bb'])
  data_tlbr_gen = xywh_to_corners(gen_data['bb'])
  for bbox_in in data_tlbr_in:
    # right now its in xywh so covert to tlbr
    cv2.rectangle(image, (int(bbox_in[0]), int(bbox_in[1])), (int(bbox_in[2]), int(bbox_in[3])),(100,255,0), 2)
  # for bbox_gen in data_tlbr_gen:
  #   cv2.rectangle(image, (int(bbox_gen[0]), int(bbox_gen[1])), (int(bbox_gen[2]), int(bbox_gen[3])),(255,0,0), 2)
  cv2_imshow(image)